In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import scipy

import tensorNets.a_mps as a_mps
import tensorNets.b_model as b_model
import tensorNets.d_dmrg as d_dmrg

from heisenbergModel import HeisenbergModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The Heisenberg model is build in `HeisenbergModel.py`. We only want to perform DMRG, so the only relevant function is `generate_H_mpo()`

rewrite Hamiltonian:
$$
H = J \sum_{i<j} \lambda ^{j-i} \sum_{\alpha = x,y,z} S_i^\alpha S_j^\alpha \qquad \lambda = e ^{-\frac{1}{\xi}}
$$

In [5]:
# hyperparameters
L = 10  # system size
J = 1.0  # coupling strength
XI = 1.0  # correlation length

In [53]:
def init_spindown_MPS(L):
    """Return a product state with all spins up as an MPS"""
    B = np.zeros([1, 2, 1], np.float64)
    B[0, 0, 0] = -1.
    S = np.ones([1], np.float64)
    Bs = [B.copy() for i in range(L)]
    Ss = [S.copy() for i in range(L)]
    return a_mps.MPS(Bs, Ss)

In [54]:
mps = init_spindown_MPS(L)
model = HeisenbergModel(L, J, XI)
engine = d_dmrg.DMRGEngine(mps, model)

In [55]:
engine.calculate_energy()

array(3.3293076)

In [59]:
model.H_mpo[0][1,1]

array([[0.36787944+0.j, 0.        +0.j],
       [0.        +0.j, 0.36787944+0.j]])

In [64]:
# function to run the DMRG algorithm

def run_dmrg_simulation(L, J, xi, max_sweeps=20, tol=1e-9):
    mps = a_mps.init_spinup_MPS(L)
    model = HeisenbergModel(L, J, xi)
    engine = d_dmrg.DMRGEngine(mps, model)

    # E_exact = tfi_exact.finite_gs_energy(L, J, g)

    E, E_prev = 0, 0

    for i in range(max_sweeps):
        engine.sweep()
        E_prev = E
        E = engine.calculate_energy()
        print("Energy:", E)
        # print("Energy error:", E - E_exact)
        if np.abs(E - E_prev) < tol:
            print(f"The system is converged after {i+1} sweeps!")
            break
    
    return model, mps, engine

In [ ]:
model, mps, engine = run_dmrg_simulation(24, J, XI, tol=1e-10)

Energy: -8.916907090342443
Energy: -8.96464826734616
Energy: -8.964964767964258
Energy: -8.96496477042869
Energy: -8.964964770428665
The system is converged after 5 sweeps!


In [68]:
engine.RPs[0]

array([[[-8.74190536e+00+2.82918907e-16j,
          4.29621931e-14+3.58197847e-14j],
        [ 3.88946128e-01-9.63482130e-19j,
          2.02388663e-01+8.23512938e-02j],
        [-7.30529411e-02+2.02750723e-17j,
         -3.80132002e-02+4.38452120e-01j],
        [ 2.05927594e-01-1.69881558e-18j,
         -3.95747170e-01+3.98602238e-16j],
        [ 1.00000000e+00-4.64256619e-18j,
         -4.68451750e-15-3.79562578e-15j]],

       [[ 4.23815640e-14-3.56888591e-14j,
         -8.74190536e+00+5.01773765e-16j],
        [ 2.02388663e-01-8.23512938e-02j,
         -3.88946128e-01+1.58235370e-17j],
        [-3.80132002e-02-4.38452120e-01j,
          7.30529411e-02+2.98762651e-17j],
        [-3.95747170e-01-3.85413084e-16j,
         -2.05927594e-01+2.75800248e-17j],
        [-4.66194552e-15+3.79095956e-15j,
          1.00000000e+00-5.36531486e-17j]]])